In [3]:
import pyodbc    
from dotenv import dotenv_values
import warnings 
warnings.filterwarnings('ignore')
from statsmodels.tools.sm_exceptions import ValueWarning

#libraries for handling data
import pandas as pd
import numpy as np 



In [4]:
# load environment variables from.env file into dictionary
environment_variables = dotenv_values(".env")
 
# get the values for the environment variables
server = environment_variables.get("server")
user = environment_variables.get("user")
password = environment_variables.get("password")
database = environment_variables.get("database")
 
# Create a database connection string using pyodbc
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={user};PWD={password}"

#Establish a connection to the database
try:
    connection = pyodbc.connect(connection_string)
    print("Connection successful")    
except Exception as e:
    print("Connection failed:", e)

Connection successful


In [5]:
cursor = connection.cursor()

# Query to get all tables in the current database
cursor.execute("""
    SELECT TABLE_NAME
    FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_TYPE = 'BASE TABLE'
""")

# Fetch and print the list of tables
tables = cursor.fetchall()
for table in tables:
    print(table[0])


Sales_July_2019
Sales_August_2019
Sales_September_2019
Sales_October_2019
Sales_November_2019
Sales_December_2019


In [6]:
# Create a cursor object
cursor = connection.cursor()

# Query to get all table names in the database 
cursor.execute("""
    SELECT TABLE_NAME
    FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_TYPE = 'BASE TABLE'
""")
tables = cursor.fetchall()

# Iterate through each table
for table in tables:
    table_name = table[0]
    print(f"Processing table: {table_name}")

    # Query to fetch all data from the table
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, connection)

    # Save the data to a CSV file
    csv_file = f"{table_name}.csv"
    df.to_csv(csv_file, index=False)
    print(f"Table {table_name} saved as {csv_file}")

Processing table: Sales_July_2019
Table Sales_July_2019 saved as Sales_July_2019.csv
Processing table: Sales_August_2019
Table Sales_August_2019 saved as Sales_August_2019.csv
Processing table: Sales_September_2019
Table Sales_September_2019 saved as Sales_September_2019.csv
Processing table: Sales_October_2019
Table Sales_October_2019 saved as Sales_October_2019.csv
Processing table: Sales_November_2019
Table Sales_November_2019 saved as Sales_November_2019.csv
Processing table: Sales_December_2019
Table Sales_December_2019 saved as Sales_December_2019.csv
